In [1]:
import Controller
from YahtzeeStatus import YahtzeeStatus
import itertools

controller = Controller.Controller()
status = YahtzeeStatus()


In [2]:
RewardTable = {}

In [3]:
import itertools
class sequence_generator:
    def __init__(self):
        self.results = {}
    def _gen(self, n):
        l = list(itertools.product(range(1, 7), repeat=n))
        m = {}
        for dices in l:
            dd = list(dices)
            dd.sort()
            tup = tuple(dd)
            if tup in m:
                m[tup] += 1
            else:
                m[tup] = 1
        return m, len(l)
    def gen_sequence(self, n=5):
        if n not in self.results:
            self.results[n] = self._gen(n)
        
        return self.results[n]

seq_gen = sequence_generator()

def CountBits(n):
    n = (n & 0x5555555555555555) + ((n & 0xAAAAAAAAAAAAAAAA) >> 1)
    n = (n & 0x3333333333333333) + ((n & 0xCCCCCCCCCCCCCCCC) >> 2)
    n = (n & 0x0F0F0F0F0F0F0F0F) + ((n & 0xF0F0F0F0F0F0F0F0) >> 4)
    n = (n & 0x00FF00FF00FF00FF) + ((n & 0xFF00FF00FF00FF00) >> 8)
    n = (n & 0x0000FFFF0000FFFF) + ((n & 0xFFFF0000FFFF0000) >> 16)
    n = (n & 0x00000000FFFFFFFF) + ((n & 0xFFFFFFFF00000000) >> 32) # This last & isn't strictly necessary.
    return n

In [7]:
def get_expect(status_code) -> float:
    if status_code in RewardTable:
        return RewardTable[status_code]

    state, dice_hash, numerical, reroll_times = YahtzeeStatus.parse_status_code(
        status_code)
    # print(state, dice_hash, numerical, reroll_times)
    dices = YahtzeeStatus.parse_dices(dice_hash)

    if CountBits(state) == len(controller.basic_scores):
        return 0
    if dice_hash > 10000:
        writables = controller.get_writable_by_statuscode(status_code)

        expect = 0
        for w in writables:
            new_status = controller.write_by_statuscode(status_code, w)

            new_expect = get_expect(
                new_status) + controller.judge_by_statuscode(status_code, w)
            if new_expect > expect:
                expect = new_expect
        if reroll_times > 0:
            for kept in range(5):
                new_dices_set = set(itertools.combinations(dices, kept))
                for dices in new_dices_set:
                    new_status = YahtzeeStatus.compose_status_code(state, YahtzeeStatus.compose_dices(dices),
                                                                   numerical, reroll_times - 1)
                    if get_expect(new_status) > expect:
                        expect = get_expect(new_status)
    else:
        kept = len(dices)
        comings, total = seq_gen.gen_sequence(5-kept)
        expect = 0
        for coming_dices in comings:
            count = comings[coming_dices]

            probability = count / len(comings)
            new_dices = list(dices) + list(coming_dices)
            new_status = YahtzeeStatus.compose_status_code(state, YahtzeeStatus.compose_dices(new_dices),
                                                           numerical, reroll_times)
            # print(new_status)
            expect += get_expect(new_status) * probability

    RewardTable[status_code] = expect
    if(len(RewardTable) % 100000 == 0):
        print(len(RewardTable))
    return expect


In [5]:
print(len(RewardTable))
print(RewardTable)

0
{}


In [8]:
init_status = 2
print(get_expect(init_status))

700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000
1800000
1900000
2000000
2100000
2200000
2300000
2400000
2500000
2600000
2700000
2800000
2900000
3000000
3100000
3200000
3300000
3400000
3500000
3600000
3700000
3800000
3900000
4000000
4100000
4200000
4300000
4400000
4500000
4600000
4700000
4800000
4900000
5000000
5100000
5200000
5300000
5400000
5500000
5600000
5700000
5800000
5900000
6000000
6100000
6200000
6300000
6400000
6500000
6600000
6700000
6800000
6900000
7000000
7100000
7200000
7300000
7400000
7500000
7600000
7700000
7800000
7900000
8000000
8100000
8200000
8300000
8400000
8500000
8600000
8700000
8800000
8900000
9000000
9100000
9200000
9300000
9400000
9500000
9600000
9700000
9800000
9900000
10000000
10100000
10200000
10300000
10400000
10500000
10600000
10700000
10800000
10900000
11000000
11100000
11200000
11300000
11400000
11500000
11600000
11700000
11800000
11900000
12000000
12100000
12200000
12300000
12400000
12500000
12600000
12700000
1280000

KeyboardInterrupt: 